In [ ]:
pip install --upgrade google-cloud-aiplatform


In [18]:
import vertexai
from vertexai.generative_models import GenerativeModel
model = GenerativeModel("gemini-1.5-pro-001")

In [ ]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

In [ ]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

In [ ]:
import requests

url = "https://scheduler-934459006621.europe-north1.run.app/graphql"
query = """
{
  employees {
    id
    name
    email
    needsChildCare
    prefersOvertime
    role
    department
    availability
    overtimePreferences
  }
}
"""
response = requests.post(url, json={'query': query})
print(response.json())

In [9]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-pro-001")

In [19]:
#get employee schedule for a specific employee

import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

# Initialize the Generative Model
model = GenerativeModel("gemini-1.5-pro-001")

# Define the GraphQL query to get the employee schedule
get_employee_schedule_func = FunctionDeclaration(
    name="get_employee_schedule",
    description="Get the schedule for a specific employee",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The schedule ID"
            },
            "employeeId": {
                "type": "string",
                "description": "The employee ID"
            }
        },
        "required": [
            "id",
            "employeeId"
        ]
    },
)

# Define the tool with the function declaration
employee_schedule_tool = Tool(
    function_declarations=[get_employee_schedule_func],
)

# Define the prompt
prompt = """Get the schedule for the employee with ID 'bob.johnson@example.com' and schedule ID '1727400263394'."""

# Generate content using the model
response = model.generate_content(
    prompt,
    tools=[employee_schedule_tool],
)

# Extract parameters from the response
params = {}
function_call = response.candidates[0].content.parts[0].function_call
if function_call is not None:
  for key, value in function_call.args.items():
    params[key] = value

# Define the GraphQL query
graphql_query = """
query {
  schedule(id: "%s", employeeId: "%s") {
    id
    employee {
      id
      name
    }
    startTime
    endTime
    isOvertime
    notes
  }
}
""" % (params['id'], params['employeeId'])

# Send the GraphQL query to the API
url = "https://scheduler-934459006621.europe-north1.run.app/graphql"
api_response = requests.post(url, json={'query': graphql_query})
schedule_data = api_response.json()

# Generate detailed content using the model
detailed_response = model.generate_content(
    [
        Content(role="user", parts=[
            Part.from_text(prompt + """ Provide detailed context and steps."""),
        ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call": {
                    "name": "get_employee_schedule",
                }
            })
        ]),
        Content(role="function", parts=[
            Part.from_function_response(
                name="get_employee_schedule",
                response={
                    "content": schedule_data,
                }
            )
        ]),
    ],
    tools=[employee_schedule_tool],
)

# Print the detailed response
print(detailed_response.candidates[0].content.parts[0].text)

The available tool can't get the schedule using employee ID.  The `get_employee_schedule` function only accepts `employeeId`, and `id` as arguments. 



In [ ]:
import requests
from vertexai.generative_models import (
  Content,
  FunctionDeclaration,
  GenerativeModel,
  Part,
  Tool,
)

# Initialize the Generative Model
model = GenerativeModel("gemini-1.5-pro-001")

# Define the GraphQL mutation to create an employee
create_employee_func = FunctionDeclaration(
  name="create_employee",
  description="Create a new employee",
  parameters={
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "The name of the employee"
      },
      "email": {
        "type": "string",
        "description": "The email of the employee"
      },
      "needsChildCare": {
        "type": "boolean",
        "description": "Whether the employee needs child care"
      },
      "prefersOvertime": {
        "type": "boolean",
        "description": "Whether the employee prefers overtime"
      },
      "role": {
        "type": "string",
        "description": "The role of the employee"
      },
      "department": {
        "type": "string",
        "description": "The department of the employee"
      },
      "availability": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "The availability of the employee"
      },
      "overtimePreferences": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "The overtime preferences of the employee"
      }
    },
    "required": [
      "name",
      "email",
      "needsChildCare",
      "prefersOvertime",
      "role",
      "department",
      "availability",
      "overtimePreferences"
    ]
  },
)

# Define the tool with the function declaration
employee_tool = Tool(
  function_declarations=[create_employee_func],
)

# Define the prompt
prompt = """Create a new employee with the following details:
Name: Ankit Singh
Email: ankit.singh@example.com
Needs Child Care: false
Prefers Overtime: true
Role: Engineer
Department: Development
Availability: ["Monday 9-5", "Wednesday 9-5"]
Overtime Preferences: ["Thursday 5-9"]
"""

# Generate content using the model
response = model.generate_content(
  prompt,
  tools=[employee_tool],
)

# Extract parameters from the response
params = {}
function_call = response.candidates[0].content.parts[0].function_call
if function_call is not None:
  for key, value in function_call.args.items():
    params[key] = value

# Define the GraphQL mutation
graphql_mutation = """
mutation {
  createEmployee(
  name: "%s",
  email: "%s",
  needsChildCare: %s,
  prefersOvertime: %s,
  role: "%s",
  department: "%s",
  availability: %s,
  overtimePreferences: %s
  ) {
  id
  name
  email
  }
}
""" % (
  params['name'],
  params['email'],
  str(params['needsChildCare']).lower(),
  str(params['prefersOvertime']).lower(),
  params['role'],
  params['department'],
  str(params['availability']).replace("'", '"'),
  str(params['overtimePreferences']).replace("'", '"')
)

# Send the GraphQL mutation to the API
url = "https://scheduler-934459006621.europe-north1.run.app/graphql"
api_response = requests.post(url, json={'query': graphql_mutation})
employee_data = api_response.json()

# Generate detailed content using the model
detailed_response = model.generate_content(
  [
    Content(role="user", parts=[
      Part.from_text(prompt + """ Provide detailed context and steps."""),
    ]),
    Content(role="function", parts=[
      Part.from_dict({
        "function_call": {
          "name": "create_employee",
        }
      })
    ]),
    Content(role="function", parts=[
      Part.from_function_response(
        name="create_employee",
        response={
          "content": employee_data,
        }
      )
    ]),
  ],
  tools=[employee_tool],
)

# Print the detailed response
if hasattr(detailed_response.candidates[0].content.parts[0], 'text'):
  print(detailed_response.candidates[0].content.parts[0].text)
else:
  print("No text attribute found in the response part.")

# Initialize the Generative Model
model = GenerativeModel("gemini-1.5-pro-001")

# Define the GraphQL mutation to create an employee
create_employee_func = FunctionDeclaration(
  name="create_employee",
  description="Create a new employee",
  parameters={
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "The name of the employee"
      },
      "email": {
        "type": "string",
        "description": "The email of the employee"
      },
      "needsChildCare": {
        "type": "boolean",
        "description": "Whether the employee needs child care"
      },
      "prefersOvertime": {
        "type": "boolean",
        "description": "Whether the employee prefers overtime"
      },
      "role": {
        "type": "string",
        "description": "The role of the employee"
      },
      "department": {
        "type": "string",
        "description": "The department of the employee"
      },
      "availability": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "The availability of the employee"
      },
      "overtimePreferences": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "The overtime preferences of the employee"
      }
    },
    "required": [
      "name",
      "email",
      "needsChildCare",
      "prefersOvertime",
      "role",
      "department",
      "availability",
      "overtimePreferences"
    ]
  },
)

# Define the tool with the function declaration
employee_tool = Tool(
  function_declarations=[create_employee_func],
)

# Define the prompt
prompt = """Create a new employee with the following details:
Name: Bob Johnson
Email: bob.johnson@example.com
Needs Child Care: false
Prefers Overtime: true
Role: Engineer
Department: Development
Availability: ["Monday 9-5", "Wednesday 9-5"]
Overtime Preferences: ["Thursday 5-9"]
"""

# Generate content using the model
response = model.generate_content(
  prompt,
  tools=[employee_tool],
)

# Extract parameters from the response
params = {}
function_call = response.candidates[0].content.parts[0].function_call
if function_call is not None:
  for key, value in function_call.args.items():
    params[key] = value

# Define the GraphQL mutation
graphql_mutation = """
mutation {
  createEmployee(
  name: "%s",
  email: "%s",
  needsChildCare: %s,
  prefersOvertime: %s,
  role: "%s",
  department: "%s",
  availability: %s,
  overtimePreferences: %s
  ) {
  id
  name
  email
  }
}
""" % (
  params['name'],
  params['email'],
  str(params['needsChildCare']).lower(),
  str(params['prefersOvertime']).lower(),
  params['role'],
  params['department'],
  str(params['availability']).replace("'", '"'),
  str(params['overtimePreferences']).replace("'", '"')
)

# Send the GraphQL mutation to the API
url = "https://scheduler-934459006621.europe-north1.run.app/graphql"
api_response = requests.post(url, json={'query': graphql_mutation})
employee_data = api_response.json()

# Generate detailed content using the model
detailed_response = model.generate_content(
  [
    Content(role="user", parts=[
      Part.from_text(prompt + """ Provide detailed context and steps."""),
    ]),
    Content(role="function", parts=[
      Part.from_dict({
        "function_call": {
          "name": "create_employee",
        }
      })
    ]),
    Content(role="function", parts=[
      Part.from_function_response(
        name="create_employee",
        response={
          "content": employee_data,
        }
      )
    ]),
  ],
  tools=[employee_tool],
)

# Print the detailed response
if hasattr(detailed_response.candidates[0].content.parts[0], 'text'):
  print(detailed_response.candidates[0].content.parts[0].text)
else:
  print("No text attribute found in the response part.")


In [ ]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

# Initialize the Generative Model
model = GenerativeModel("gemini-1.5-pro-001")

# Define the GraphQL mutation to create an employee
create_employee_func = FunctionDeclaration(
    name="create_employee",
    description="Create a new employee",
    parameters={
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the employee"
            },
            "email": {
                "type": "string",
                "description": "The email of the employee"
            },
            "needsChildCare": {
                "type": "boolean",
                "description": "Whether the employee needs child care"
            },
            "prefersOvertime": {
                "type": "boolean",
                "description": "Whether the employee prefers overtime"
            },
            "role": {
                "type": "string",
                "description": "The role of the employee"
            },
            "department": {
                "type": "string",
                "description": "The department of the employee"
            },
            "availability": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "description": "The availability of the employee"
            },
            "overtimePreferences": {
                "type": "array",
                "items": {
                    "type": "string"
                },
                "description": "The overtime preferences of the employee"
            }
        },
        "required": [
            "name",
            "email",
            "needsChildCare",
            "prefersOvertime",
            "role",
            "department",
            "availability",
            "overtimePreferences"
        ]
    },
)

# Define the tool with the function declaration
employee_tool = Tool(
    function_declarations=[create_employee_func],
)

# Define the prompt
prompt = """Create a new employee with the following details:
Name: Bob Johnson
Email: bob.johnson@example.com
Needs Child Care: false
Prefers Overtime: true
Role: Engineer
Department: Development
Availability: ["Monday 9-5", "Wednesday 9-5"]
Overtime Preferences: ["Thursday 5-9"]
"""

# Generate content using the model
response = model.generate_content(
    prompt,
    tools=[employee_tool],
)

# Extract parameters from the response
params = {}
function_call = response.candidates[0].content.parts[0].function_call
if function_call is not None:
    for key, value in function_call.args.items():
        params[key] = value

# Define the GraphQL mutation
graphql_mutation = """
mutation {
  createEmployee(
    name: "%s",
    email: "%s",
    needsChildCare: %s,
    prefersOvertime: %s,
    role: "%s",
    department: "%s",
    availability: %s,
    overtimePreferences: %s
  ) {
    id
    name
    email
  }
}
""" % (
    params['name'],
    params['email'],
    str(params['needsChildCare']).lower(),
    str(params['prefersOvertime']).lower(),
    params['role'],
    params['department'],
    str(params['availability']).replace("'", '"'),
    str(params['overtimePreferences']).replace("'", '"')
)

# Send the GraphQL mutation to the API
url = "https://scheduler-934459006621.europe-north1.run.app/graphql"
api_response = requests.post(url, json={'query': graphql_mutation})
employee_data = api_response.json()

# Generate detailed content using the model
detailed_response = model.generate_content(
    [
        Content(role="user", parts=[
            Part.from_text(prompt + """ Provide detailed context and steps."""),
        ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call": {
                    "name": "create_employee",
                }
            })
        ]),
        Content(role="function", parts=[
            Part.from_function_response(
                name="create_employee",
                response={
                    "content": employee_data,
                }
            )
        ]),
    ],
    tools=[employee_tool],
)

# Print the detailed response
print(detailed_response.candidates[0].content.parts[0].text)